In [1]:
import pandas as pd
import numpy as np
import os, sys, regex

In [2]:
dala = os.getenv('HOME')

In [6]:
base_dir = f'{dala}/Documents/RunSheets/AllRunSheets'

In [7]:
cols1 = ['Sample_Name', 'Reporter', 'CT', 'Ct_Mean']

data = pd.DataFrame(columns=cols1)
data_list = []
for file in os.listdir(base_dir):
    if file.endswith('.xls'):
        data_list.append(file)
    pass
sheets = sorted(data_list)

In [8]:
len(np.unique(np.array(sheets)))

476

In [9]:
' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2])

'2020-10-29 15:46:45'

In [10]:
pd.to_datetime(' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2]), yearfirst=True)

Timestamp('2020-10-29 15:46:45')

In [11]:
cols2 = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean']

counts = 0

for sheet in sheets:
    df = pd.read_excel(f'{base_dir}/{sheet}')
    count2 = 0
#     print(sheet)
    for row in df.itertuples():
        count2 += 1
        if row[1] == 'Experiment Run End Time':
            expt_run_date = pd.to_datetime(' '.join(row[2].split(' ')[:-2]), yearfirst=True)
        elif row[1] == 'Well':
            df = pd.read_excel(f'{base_dir}/{sheet}', 'Results', header=count2)
            cols3 = list(set(df.columns) & set(cols2))
#             print(cols3)
#             print(list(df.columns))
            df2 = df[sorted(cols3)]
            df2.columns = sorted(cols1)
            df3 = df2.assign(Dt_Run=expt_run_date)
            data = data.append(df3, ignore_index=True)
            counts += 1
            break
        else: pass

In [12]:
print(f'number of sheets in folder: {len(sheets)}')
print(f'data rows: {data.shape[0]}')
print(f'sheets processed: {counts}')

number of sheets in folder: 476
data rows: 96338
sheets processed: 476


In [13]:
data.head()

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
0,COVC 13541,FAM,Undetermined,NaN,2020-10-12 15:33:47
1,COVC 13541,VIC,Undetermined,NaN,2020-10-12 15:33:47
2,COVC 13541,CY5,21.914249,21.914249,2020-10-12 15:33:47
3,COVC 13542,FAM,Undetermined,NaN,2020-10-12 15:33:47
4,COVC 13542,VIC,Undetermined,NaN,2020-10-12 15:33:47


In [14]:
data.info()#.nunique()#.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96338 entries, 0 to 96337
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Sample_Name  94859 non-null  object        
 1   Reporter     95057 non-null  object        
 2   CT           94766 non-null  object        
 3   Ct_Mean      44674 non-null  float64       
 4   Dt_Run       95238 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 3.7+ MB


In [15]:
# undetermined_ids = list(data[data['CT'] == 'Undetermined']['Sample_Name'].unique())#.astype(list)

In [16]:
data2 = data[data['CT'] != 'Undetermined']

In [17]:
data3 = data2[data2['Reporter'].isin(['FAM', 'VIC', 'ABY']) == True]

In [18]:
data1 = data3[data3['Sample_Name'].isin(['PC', 'NC']) != True]

In [19]:
data4 = data1[data1.duplicated('Sample_Name', False) == True]

In [20]:
data4.head(10)

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
126,COVC 13583,FAM,37.328472,37.328472,2020-10-12 15:33:47
127,COVC 13583,VIC,42.683975,42.683975,2020-10-12 15:33:47
135,COVC 13586,FAM,35.068436,35.068436,2020-10-12 15:33:47
136,COVC 13586,VIC,39.385445,39.385445,2020-10-12 15:33:47
186,COVC 13603,FAM,27.376181,27.376181,2020-10-12 15:33:47
187,COVC 13603,VIC,31.060112,31.060112,2020-10-12 15:33:47
219,COVC 13614,FAM,36.698231,36.698231,2020-10-12 15:33:47
231,COVC 13618,FAM,32.132778,32.132778,2020-10-12 15:33:47
232,COVC 13618,VIC,36.225323,36.225323,2020-10-12 15:33:47
261,COVC 13628,FAM,33.411869,33.411869,2020-10-12 15:33:47


In [21]:
def get_upper_alpha_seq():
    letters = []
    for c in range(65, 91):
        letters.append(chr(c))
    return letters

In [22]:
def cov_rename(x):
    try:
        if 'COV' not in x and '-R' in x:
            x = x.replace('-R', '')
        if 'COV' in x and 'COVC' not in x:
            x = x.replace('COV', 'COVC')
        int(x)
        x = str(x).replace(str(x), f'COVC{str(x)}')
    except (ValueError, AttributeError, TypeError): x = str(x).replace(' ', '')
    return x

In [23]:
def cov_names(x):
    cov_id = str(x).upper().replace(' ', '').rstrip(f'#-_{get_upper_alpha_seq()}')
    if len(cov_id) == 8:
        return cov_id.replace('COVC', 'COVC0')
    elif len(cov_id) == 7:
        return cov_id.replace('COVC', 'COVC00')
    elif len(cov_id) == 6:
        return cov_id.replace('COVC', 'COVC000')
    elif len(cov_id) == 5:
        return cov_id.replace('COVC', 'COVC0000')
    return cov_id

In [24]:
data4['Reporter'].unique()

array(['FAM', 'VIC', 'ABY'], dtype=object)

In [25]:
data5 = data4.assign(Sample_Name=data4['Sample_Name'].apply(lambda x: cov_names(cov_rename(x))))

In [26]:
data5.head()

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
126,COVC13583,FAM,37.328472,37.328472,2020-10-12 15:33:47
127,COVC13583,VIC,42.683975,42.683975,2020-10-12 15:33:47
135,COVC13586,FAM,35.068436,35.068436,2020-10-12 15:33:47
136,COVC13586,VIC,39.385445,39.385445,2020-10-12 15:33:47
186,COVC13603,FAM,27.376181,27.376181,2020-10-12 15:33:47


In [27]:
(data5.sort_values(['Sample_Name', 'Reporter', 'Dt_Run'])
 .drop_duplicates(['Sample_Name', 'Reporter'], keep='last', inplace=True))

In [28]:
data5[data5['Sample_Name'].str.contains('COVC') == False]#['Dt_Run'].unique()

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
2505,,FAM,39.200447,39.200447,2020-10-14 21:36:34
2508,,FAM,30.585463,30.585463,2020-10-14 21:36:34
2509,,VIC,30.394487,30.394487,2020-10-14 21:36:34
4074,,FAM,30.16593,30.165930,2020-10-10 15:10:47
4075,,VIC,31.120264,31.120264,2020-10-10 15:10:47
...,...,...,...,...,...
93388,C3,VIC,35.654247,34.974087,2021-06-30 18:48:52
93393,E5,FAM,33.77232,32.318531,2021-06-30 18:48:52
93394,E5,VIC,37.229836,35.226273,2021-06-30 18:48:52
93396,,FAM,31.103909,29.397940,2021-06-30 18:48:52


In [29]:
data6 = data5.sort_values(['Sample_Name']).groupby('Sample_Name').mean()

In [30]:
data6.head()

,Ct_Mean
Sample_Name,
,28.180588
A1,33.064911
C3,33.114729
COVC00137,37.646282
COVC00154,35.240269


In [31]:
df_cts = data6.reset_index()[data6.reset_index()['Sample_Name'].str.contains('COV') == True]

In [32]:
df_cts

,Sample_Name,Ct_Mean
3,COVC00137,37.646282
4,COVC00154,35.240269
5,COVC00744,34.715912
6,COVC00762,36.049999
7,COVC00767,34.074465
...,...,...
4268,COVC_8176,35.033876
4269,COVC_8177,25.184707
4270,COVC_8191,23.228168
4271,COVC_8193,26.923361


In [33]:
df_cts.to_excel('/home/douso/Documents/RunSheets/Cts/runsheet-cts.xlsx', index=False, float_format='%.1f')